# Demonstration of basic image manipulation with SIRF
This demonstration shows how to read images and data, display them.

This demo is a jupyter notebook, i.e. intended to be run step by step.
You could export it as a Python file and run it one go, but that might
make little sense as the figures are not labelled.

This notebook constitutes the first half of the PET [display_and_projection](../PET/display_and_projection.ipynb) notebook.

Author: Kris Thielemans, Richard Brown
First version: 8th of September 2016  
Second Version: 17th of May 2018
Third Version: 23rd of October 2019

CCP PETMR Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2019 University College London.

This is software developed for the Collaborative Computational
Project in Positron Emission Tomography and Magnetic Resonance imaging
(http://www.ccppetmr.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Initial set-up

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
# Make sure everything is installed that we need
!pip install brainweb nibabel --user

In [ ]:
#%% Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil
import brainweb
from tqdm.auto import tqdm

from sirf.Utilities import examples_data_path

# Utilities

In [ ]:
#%% First define some handy function definitions
# To make subsequent code cleaner, we have a few functions here. You can ignore
# ignore them when you first see this demo.
# They have (minimal) documentation using Python docstrings such that you 
# can do for instance "help(subplot_)"
#

def subplot_(idx,vol,title,clims=None,cmap="viridis"):
    """Customized version of subplot"""
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar()
    plt.title(title)
    plt.axis("off")

def crop_and_fill(templ_im, vol):
    """Crop volumetric image data and replace image content in template image object"""
    # Get size of template image and crop
    #idim = templ_im.dimensions() # TODO
    idim_orig = templ_im.as_array().shape
    idim = (1,)*(3-len(idim_orig)) + idim_orig
    offset = (numpy.array(vol.shape) - numpy.array(idim)) // 2
    vol = vol[offset[0]:offset[0]+idim[0], offset[1]:offset[1]+idim[1], offset[2]:offset[2]+idim[2]]
    # Fill image content
    templ_im.fill(numpy.reshape(vol, idim_orig))
    return(templ_im)

# Get brainweb data

In [ ]:
fname, url= sorted(brainweb.utils.LINKS.items())[0]
files = brainweb.get_file(fname, url, ".")
data = brainweb.load_file(fname)

brainweb.seed(1337)

In [ ]:
for f in tqdm([fname], desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(f, petNoise=1, t1Noise=0.75, t2Noise=0.75, petSigma=1, t1Sigma=1, t2Sigma=1)

In [ ]:
FDG_arr  = vol['PET']
T1_arr   = vol['T1']
uMap_arr = vol['uMap']

In [ ]:
#%% Display it
plt.figure();
slice_show = FDG_arr.shape[0]//2
subplot_([1,3,1], FDG_arr[slice_show, 100:-100, 100:-100], 'FDG', cmap="hot")
subplot_([1,3,2], T1_arr[slice_show, 100:-100, 100:-100], 'T1', cmap="Greys_r")
subplot_([1,3,3], uMap_arr[slice_show, 100:-100, 100:-100], 'uMap', cmap="bone")

# MR

In [ ]:
#%% Use the 'mr' prefix for all Gadgetron-based SIRF functions.
# This is done here to explicitly differentiate between SIRF mr functions and 
# anything else.
import sirf.Gadgetron as mr

In [ ]:
# We'll need a template MR acquisition data object
templ_mr = mr.AcquisitionData(examples_data_path('MR') + '/simulated_MR_2D_cartesian.h5')

In [ ]:
# Now we can create a template MR image data
#im_mr = mr.ImageData(templ_mr)

# TODO 
# -> work around
preprocessed_data = mr.preprocess_acquisition_data(templ_mr)
recon = mr.FullySampledReconstructor()
recon.set_input(preprocessed_data)
recon.process()
im_mr = recon.get_output()

Now we have got an MR image object and can fill it with the brainweb data. The dimensions won't fit, but we will simply crop the image.

In [ ]:
im_mr = crop_and_fill(im_mr, T1_arr)
plt.figure();
subplot_([1,1,1], numpy.abs(im_mr.as_array())[im_mr.dimensions()[0]//2, :, :], 'MR', cmap="Greys_r")

# CT

In [ ]:
#%% Use the 'ct' prefix for all CIL-based SIRF functions.
# This is done here to explicitly differentiate between SIRF ct functions and 
# anything else.
import cil.framework as ct #import ImageGeometry, AcquisitionGeometry

In [ ]:
#%% Create a template CT acquisition geometry
N = 120
angles = numpy.linspace(0, 360, 50, True, dtype=numpy.float32)
offset = 0.4
channels = 1
ag = ct.AcquisitionGeometry.create_Cone3D((offset,-100, 0), (offset,100,0))
ag.set_panel((N,N-2))
ag.set_channels(channels)
ag.set_angles(angles, angle_unit=ct.AcquisitionGeometry.DEGREE)

In [ ]:
# Now we can create a template CT image object
ig = ag.get_ImageGeometry()
im_ct = ig.allocate(None)

Now we have got an CT image object and can fill it with the brainweb data. The dimensions won't fit, but we will simply crop the image.

In [ ]:
im_ct = crop_and_fill(im_ct, uMap_arr)

plt.figure();
subplot_([1,1,1], im_ct.as_array()[im_ct.as_array().shape[0]//2, :, :], 'CT', cmap="bone")

# PET

In [ ]:
#%% Use the 'pet' prefix for all STIR-based SIRF functions.
# This is done here to explicitly differentiate between SIRF pet functions and 
# anything else.
import sirf.STIR as pet

In [ ]:
# We'll need a template sinogram
templ_sino = pet.AcquisitionData(examples_data_path('PET') + "/mMR/mMR_template_span11.hs")

In [ ]:
# Now we can create a template PET image object
im_pet = pet.ImageData(templ_sino)

Now we have got an PET image object and can fill it with the brainweb data. The dimensions won't fit, but we will simply crop the image.

In [ ]:
im_pet = crop_and_fill(im_pet, FDG_arr)

plt.figure();
subplot_([1,1,1], im_pet.as_array()[im_pet.dimensions()[0]//2, :, :], 'PET', cmap="hot")

# Basic image manipulations
Images (like most other things in SIRF) are represented as *objects*, in this case of type `ImageData`.
In practice, this means that you can only manipulate its data via *methods*.

Image objects contain the actual voxel values, but also information on the number of voxels,
voxel size, etc. There are methods to get this information.

There are additional methods for other manipulations, such as basic image arithmetic (e.g.,
you can add image objects).

In [ ]:
#%% Read in images
# Here we will read some images provided with the demo using the ImageData class.
# These are in Interfile format. (A text header pointing to a .v file with the binary data).
image = im_ct

In [ ]:
#%% What is an ImageData?
# Images are represented by objects with several methods. The most important method 
# is as_array() which we'll use below.
# Let's see what all the methods are.
help(pet.ImageData)

In [ ]:
#%% Use as_array to extract an array of voxel values
# The resulting array as a `numpy` array, as standard in Python.
image_array=image.as_array()
# We can use the standard `numpy` methods on this array, such as getting its `shape` (i.e. dimensions).
print(image_array.shape)
# Whenever we want to do something with the image-values, we have to do it via this array.
# Let's print a voxel-value roughly in the centre of the object. We can't use the centre because this happens to be 0
centre = numpy.array(image_array.shape)//2
print(image_array[centre[0], centre[1]+20, centre[2]+20])

In [ ]:
#%% Manipulate the image data for illustration
# Multiply the data with a factor
image_array *= 0.01
# Stick this new data into the original image object.
# (This will not modify the file content, only the variable in memory.)
image.fill(image_array)
print(image_array[centre[0], centre[1]+20, centre[2]+20])

In [ ]:
#%% You can do basic math manipulations with ImageData objects 
# So the above lines can be done directly on the `image` object
image *= 0.01
# Let's check
image_array=image.as_array()
print(image_array[centre[0], centre[1]+20, centre[2]+20])

In [ ]:
#%% Display the middle slice of the image (which is really a 3D volume)
# We will use our own imshow function (which was defined above) for brevity.

# Create a new figure
plt.figure()
# Display the slice
subplot_([1,1,1], image_array[centre[0], 0:-1, 0:-1], 'image data', cmap="viridis")

In [ ]:
#%% Some other things to do with ImageData objects
print(image.voxel_sizes())
another_image=image.clone()
an_image_with_fixed_values = image.get_uniform_copy(5)